In [13]:
# pip install openpyxl

In [14]:
import pandas as pd
# import openpyxl

In [15]:
sumula = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Sumula_Novo_Padrao.xlsx",
    sheet_name=None,
    header=1,
    engine="calamine"
)

In [16]:
# sumula["Individual"] = sumula["Individual"][sumula["Individual"]["Boleiro"].notna()]
sumula["Individual"] = sumula["Individual"].query("Boleiro == Boleiro") # Mesmo efeito do notna()

sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                      "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]] = \
    sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                          "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]].fillna(0).astype("int")
# sumula["Individual"]

In [17]:
print(sumula.keys())

dict_keys(['Baba', 'Boleiros', 'Individual', 'Times', 'Pizza', 'Premios', 'DefesasPenaltis', 'Ouros', 'Contra', 'Cartoes'])


In [18]:
lista_resultados = []
sumula["Pizza"]["Pizza"] = 1
for col in sumula["Pizza"].columns:
    if col.startswith("Colete"):
        resultados = sumula["Individual"].merge(sumula["Pizza"], how="left", left_on=["Baba", "Colete"], right_on=["Baba", col])
        lista_resultados.append(resultados)
sumula_pizza = pd.concat(lista_resultados, ignore_index=True)
sumula_pizza["Pizza"] = sumula_pizza["Pizza"].fillna(0).astype(int)

sumula_pizza = sumula_pizza.drop(
    columns=[col for col in sumula_pizza.columns if col.startswith("Colete") and col != "Colete"]
)

# Quando há mais de um "colete" vencedor, os resultados são duplicados, sendo necessário remover as duplicatas
sumula_pizza = sumula_pizza.sort_values(by="Pizza", ascending=False).drop_duplicates(subset=["Baba", "Boleiro"], keep="first")

sumula_pizza = sumula_pizza.sort_values(by=["Baba", "Colete", "Boleiro"], ascending=True).reset_index(drop=True)

In [19]:
sumula_defesaspenalti = sumula_pizza.merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
sumula_defesaspenalti["Defesas de Penalti"] = sumula_defesaspenalti["Defesas de Penalti"].fillna(0).astype("int")
sumula_defesaspenalti

,Baba,Boleiro,Colete,Gols,Assistencias,Vitorias,Derrotas,Empates,Pontos no Baba,Mensalidade,Colaboracao,Pontos de Copa,Pizza,Defesas de Penalti
0,T10B01,Anderson,Verde,1,3,3,5,1,15,0,0,0,0,0
1,T10B01,André,Verde,0,2,3,5,1,13,0,0,0,0,0
2,T10B01,Carlos,Verde,4,0,3,5,1,15,3,0,0,0,0
3,T10B01,João,Verde,1,1,3,5,1,13,0,0,0,0,0
4,T10B01,Matheus,Verde,2,2,3,5,1,15,3,10,0,0,1
5,T10B01,Júlio,Vermelho,2,1,5,3,1,20,0,0,5,1,0
6,T10B01,Leandro,Vermelho,1,0,5,3,1,18,0,0,0,1,0
7,T10B01,Pedro,Vermelho,4,1,5,3,1,22,0,0,0,1,0
8,T10B01,Renato,Vermelho,2,1,5,3,1,20,0,10,0,1,0
9,T10B01,Zé,Vermelho,4,4,5,3,1,25,3,0,0,1,0


In [ ]:
# Ouros (O), Gols Contra (C), Cartões (C)
sumula_occ = (sumula_defesaspenalti
              .merge(sumula["Ouros"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Contra"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Cartoes"], how="left", on=["Baba", "Boleiro"])
              )
sumula_occ[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]] = \
sumula_occ[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]].fillna(0).astype("int")
sumula_occ

,Baba,Boleiro,Colete,Gols,Assistencias,Vitorias,Derrotas,Empates,Pontos no Baba,Mensalidade,Colaboracao,Pontos de Copa,Pizza,Defesas de Penalti,Ouros,Gols Contra,Cartao Amarelo,Cartao Azul,Cartao Vermelho
0,T10B01,Anderson,Verde,1,3,3,5,1,15,0,0,0,0,0,0,0,1,0,0
1,T10B01,André,Verde,0,2,3,5,1,13,0,0,0,0,0,0,0,0,0,0
2,T10B01,Carlos,Verde,4,0,3,5,1,15,3,0,0,0,0,0,0,0,0,0
3,T10B01,João,Verde,1,1,3,5,1,13,0,0,0,0,0,0,0,0,0,0
4,T10B01,Matheus,Verde,2,2,3,5,1,15,3,10,0,0,1,0,0,0,0,0
5,T10B01,Júlio,Vermelho,2,1,5,3,1,20,0,0,5,1,0,0,0,0,0,0
6,T10B01,Leandro,Vermelho,1,0,5,3,1,18,0,0,0,1,0,1,0,0,0,0
7,T10B01,Pedro,Vermelho,4,1,5,3,1,22,0,0,0,1,0,0,0,0,0,0
8,T10B01,Renato,Vermelho,2,1,5,3,1,20,0,10,0,1,0,0,0,0,0,0
9,T10B01,Zé,Vermelho,4,4,5,3,1,25,3,0,0,1,0,0,0,2,1,0


In [21]:
# duplas_final = pd.DataFrame()
# for coluna_boleiro in time.columns:
#     if "Boleiro" in coluna_boleiro:
#         duplas = (sumula[["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"]]
#                 .merge(time, how="left", left_on=["Baba", "Colete", "Boleiro"], right_on=["Baba", "Colete", coluna_boleiro])
#                 .melt(id_vars=["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"],
#             value_vars=["Boleiro1", "Boleiro2", "Boleiro3", "Boleiro4", "Boleiro5", "Boleiro6"],
#             var_name="Origem",
#             value_name="SegundoBoleiro")
#             .drop(columns="Origem"))
#         duplas = duplas[duplas["Boleiro"] != duplas["SegundoBoleiro"]].reset_index(drop=True)
#         duplas_final = pd.concat([duplas_final, duplas], ignore_index=True)
# duplas_final = duplas_final.rename(columns={"SegundoBoleiro":"Boleiro2"})

In [22]:
# grupo_duplas = duplas_final.groupby(["Boleiro", "Boleiro2"]).agg(
#     Total_Pizza=("Pizza", "sum"),
#     Total_Vitorias=("Vitorias", "sum"),
#     Total_Derrotas=("Derrotas", "sum"),
#     Total_Empates=("Empates", "sum"),
#     Total_Babas=("Baba", "count")
# ).reset_index()
# grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
# grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
# grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
# grupo_duplas["M_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Babas"]
# grupo_duplas["M_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Babas"]
# grupo_duplas = grupo_duplas[
#             [
#                 "Boleiro",
#                 "Boleiro2",
#                 "Total_Pizza",
#                 "Total_Vitorias",
#                 "Total_Derrotas",
#                 "Total_Empates",
#                 "Total_Partidas",
#                 "Total_Babas",
#                 "P_Vitorias",
#                 "P_Derrotas",
#                 "P_Empates",
#                 "P_VD",
#                 "M_Pizza",
#                 "M_Vitorias",
#                 "M_Derrotas",
#                 "M_Empates"
#             ]
#         ]

In [23]:
# grupo_duplas = grupo_duplas.sort_values(["Total_Babas", "Total_Partidas", "Total_Vitorias"], ascending=False).reset_index(drop=True)
# grupo_duplas.index = grupo_duplas.index + 1
# grupo_duplas.to_excel("grupo_duplas.xlsx", index=False)
# grupo_duplas

In [24]:
# ranking_completo = sumula.groupby("Boleiro").agg(
#     Artilharia=("Gols", "sum"),
#     Assistencias=("Assistencias", "sum"),
#     Babas_Conquistados=("Vitorias", "sum"),
#     Babas_Perdidos=("Derrotas", "sum"),
#     Empates=("Empates", "sum"),
#     Pizza=("Pizza", "sum"),
#     Presenca_no_Baba=("Baba", "count"),
#     PB=("Pontos no Baba", "sum"),
#     Mensalidade=("Mensalidade", "sum"),
#     Pontos_de_Colaboracao=("Colaboracao","sum"),
#     Copa=("Pontos de Copa","sum"),
#     Ouros=("Ouros","sum"),
#     Gols_Contra=("Gols Contra","sum"),
#     ).reset_index()
# # ranking_completo